In [1]:
path = "https://raw.githubusercontent.com/shobhit-nigam/knowledgeclan/main/datasets/income.csv"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

In [3]:
dfa = pd.read_csv(path)

In [4]:
str_cols = ['workclass', 'education','marital_status', 'occupation','relationship','race',
 'sex','native_country','high_income']

In [5]:
for name in str_cols:
    col = pd.Categorical(dfa[name])
    dfa[name] = col.codes

In [6]:
dfa = dfa.sample(frac=1, random_state=1)    #num= len(dfa)

In [7]:
list_cols = ['age', 'workclass', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'hours_per_week', 'native_country']

In [8]:
train_max_row = round(dfa.shape[0] * 0.8)
train = dfa.iloc[:train_max_row]
test = dfa.iloc[train_max_row:]

In [13]:
model_a = DecisionTreeClassifier(random_state=1, min_samples_leaf=2)
model_a.fit(train[list_cols], train["high_income"])

test_predictions_a = model_a.predict(test[list_cols])
test_auc_a = roc_auc_score(test["high_income"], test_predictions_a)

print(test_auc_a)

0.68788622088561


In [14]:
model_b = DecisionTreeClassifier(random_state=1, max_depth=5)
model_b.fit(train[list_cols], train["high_income"])

test_predictions_b = model_b.predict(test[list_cols])
test_auc_b = roc_auc_score(test["high_income"], test_predictions_b)

print(test_auc_b)

0.6759810785833998


In [16]:
model_a = DecisionTreeClassifier(random_state=1, min_samples_leaf=2)
model_a.fit(train[list_cols], train["high_income"])

test_predictions_a = model_a.predict_proba(test[list_cols])

In [18]:
model_b = DecisionTreeClassifier(random_state=1, max_depth=5)
model_b.fit(train[list_cols], train["high_income"])

test_predictions_b = model_b.predict_proba(test[list_cols])

In [20]:
combination = np.round((test_predictions_a[:, 1] + test_predictions_b[:, 1])/2 )

In [21]:
combination_auc = roc_auc_score(test["high_income"], combination)

print(combination_auc)

0.7150746048901211


In [22]:
tree_count = 10
bag_ratio = 6/10  # (60%)
predictions = []

for i in range(tree_count):
    bag = train.sample(frac=bag_ratio, replace=True, random_state=i)
    
    model = DecisionTreeClassifier(min_samples_leaf=2, random_state=1)
    model.fit(bag[list_cols], bag["high_income"])
    
    predictions.append(model.predict_proba(test[list_cols])[:, 1])
    combination = np.round((np.sum(predictions, axis=0))/(len(predictions)))
    
print(roc_auc_score(test["high_income"], combination))

0.7325780430117632


In [24]:
tree_count = 10
bag_ratio = 6/10  # (60%)
predictions = []

for i in range(tree_count):
    bag = train.sample(frac=bag_ratio, replace=True, random_state=i)
    
    model = DecisionTreeClassifier(max_depth=13, random_state=1)
    model.fit(bag[list_cols], bag["high_income"])
    
    predictions.append(model.predict_proba(test[list_cols])[:, 1])
    combination = np.round((np.sum(predictions, axis=0))/(len(predictions)))
    
print(roc_auc_score(test["high_income"], combination)) 

0.7393292295161569
